# Gerador de arquivos com base no Robust04

## Installs

In [ ]:
!pip install jsonlines

In [ ]:
!pip install fpdf

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=ec071f48321f313509f51541cc51b3457ce8225d7d2597eb94af1ecc09e610be
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built fpdf


In [ ]:
!pip install python-docx 

     |████████████████████████████████| 5.6 MB 7.2 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=363c8ae6c525e18c84d4bf06cb02a353e8c3ebaa3db0ce54fdad9cd7602e00e8
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
!pip install docx2pdf

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 1.7.0 which is incompatible.


## Imports

In [ ]:
import jsonlines
from tqdm import tqdm
import re, os
from fpdf import FPDF

from docx import Document
from docx.shared import Inches
from docx2pdf import convert

## Login GCS

In [ ]:
!gcloud auth login
!gcloud auth application-default login

## Obter dataset Robust04

In [ ]:
!mkdir data
%cd data
!mkdir robust04
%cd robust04

!wget https://storage.googleapis.com/castorini/robust04/trec_disks_4_and_5_concat.txt
%cd ../../

/content/data
/content/data/robust04
--2021-12-08 22:29:28--  https://storage.googleapis.com/castorini/robust04/trec_disks_4_and_5_concat.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 74.125.31.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1997014502 (1.9G) [text/plain]
Saving to: ‘trec_disks_4_and_5_concat.txt’

trec_disks_4_and_5_ 100%[===================>]   1.86G   104MB/s    in 19s     

2021-12-08 22:29:47 (99.7 MB/s) - ‘trec_disks_4_and_5_concat.txt’ saved [1997014502/1997014502]

/content


## Função para carregar o Corpus

In [ ]:
def load_corpus(path, html=False):
    print('Loading corpus...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = f.read()
        for raw_text in tqdm(all_text.split('</DOC>')):
            if not raw_text:
                continue
            result = re.search(r'\<DOCNO\>(.*)\<\/DOCNO\>\n', raw_text)
            if not result:
                continue
            doc_id = result.group(1)
            
            doc_id = doc_id.replace('0','A').replace('1','B').replace('2','C').replace('3','D').replace('4','E')
            doc_id = doc_id.replace('5','F').replace('6','G').replace('7','H').replace('8','I').replace('9','J')
            doc_id = doc_id.replace('-','M')
            
            doc_id = doc_id.strip()
            raw_text = ' '.join(raw_text.split())

            result = re.search(r'\<TEXT\>(.*)\<\/TEXT\>', raw_text)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                if not html:
                    doc_text = doc_text.replace('<P>', ' ').replace('</P>', ' ')
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1
                    
            corpus[doc_id] = ' '.join(doc_text.split())
            
            # break

    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

## Carregando o Corpus

In [ ]:
corpus = load_corpus('/content/data/robust04/trec_disks_4_and_5_concat.txt', False)

Loading corpus...


100%|██████████| 528160/528160 [00:57<00:00, 9111.76it/s]


Loaded 528155 docs, 523869 with texts.


## Testes

In [ ]:
print(len(corpus.keys()))

528155


In [ ]:
corpus['FTJDBMBGGFF']

"REDLAND, the building materials group which paid Pounds 624m in shares for Steetley last year, has raised Pounds 100m through disposals. It is reinvesting Pounds 58.6m in its continental European tile and brick activities. Redland is forming a joint venture with Koramic, a private Belgian company, encompassing the two companies' brick making in the Netherlands, Germany and Belgium. The new company will be the largest facing brick maker in continental Europe. Profits of the combined activities in 1992 would have been Pounds 15.9m, of which Redland's share would have been Pounds 5.8m. The 50:50 joint venture will own 70 per cent of the combined activities, giving Redland a 35 per cent economic interest but joint control. Redland will also receive Pounds 17.3m in cash. Redland is increasing its stake in Coverland, the French roof tile producer, from 43 per cent to 67 per cent by acquiring part of St Gobain's shareholding. Options for Redland to purchase St Gobain's remaining 33 per cent 

## Teste Decoder
Para o nome dos arquivos

In [ ]:
print(f'Decode the file name:\n')

for i, k in enumerate(corpus.keys()):
    
    doc_id = k[2:]
    
    doc_id = doc_id.replace('A','0').replace('B','1').replace('C','2').replace('D','3').replace('E','4')
    doc_id = doc_id.replace('F','5').replace('G','6').replace('H','7').replace('I','8').replace('J','9')
    doc_id = doc_id.replace('M','-')
    
    doc_id = k[:2] + doc_id
    
    print(f'{k} : {doc_id}')
    
    if i == 9:
        break
        
print(f'\nLength of Corpus: {len(corpus.keys())}')

## Gerador de Documentos

### Gerador .docx

In [ ]:
%cd data
!mkdir docx
%cd ../

/content/data
/


In [ ]:
for key, values in corpus.items():
    
    document = Document()
    p = document.add_paragraph(values)
    document.save("/content/data/docx/" + str(key) + '.docx')

In [ ]:
with open('/content/data/sitemap.xml','w') as writer:    
  
    writer.write('<?xml version="1.0" encoding="UTF-8"?>\n\n')
    writer.write('<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n\n')

    for key, values in corpus.items():
        writer.write('\t<url>\n')
        writer.write('\t\t<loc>https://storage.googleapis.com/robust04-public/' + str(key) + '.docx</loc>\n')
        writer.write('\t\t<lastmod>2021-11-25</lastmod>\n')
        writer.write('\t</url>\n\n')
        
    writer.write('\t</urlset>\n\n')

In [ ]:
!gsutil -m cp /content/docx/* gs://robust04-public/

In [ ]:
!gsutil rm -r gs://robust04-public/sitemap.xml
!gsutil cp /content/data/sitemap.xml gs://robust04-public/

### Gerador .pdf

Só funciona em sistema que possui o Microsoft Word

In [ ]:
%cd data
!mkdir pdf
%cd ../

/content/data
/content
data  sample_data


In [ ]:
for key, values in corpus.items():
    
    convert("/content/data/docx/" + values + ".docx", "/content/data/pdf/" + values + ".pdf")

In [ ]:
with open('/content/data/sitemap.xml','w') as writer:    
  
    writer.write('<?xml version="1.0" encoding="UTF-8"?>\n\n')
    writer.write('<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n\n')

    for key, values in corpus.items():
        writer.write('\t<url>\n')
        writer.write('\t\t<loc>https://storage.googleapis.com/robust04-public/' + str(key) + '.pdf</loc>\n')
        writer.write('\t\t<lastmod>2021-11-25</lastmod>\n')
        writer.write('\t</url>\n\n')
        
    writer.write('\t</urlset>\n\n')

In [ ]:
!gsutil -m cp /content/docx/* gs://robust04-public/

In [ ]:
!gsutil rm -r gs://robust04-public/sitemap.xml
!gsutil cp /content/data/sitemap.xml gs://robust04-public/

### Gerador .html

In [ ]:
%cd data
!mkdir html
%cd ../

/content/data
/content


In [ ]:
for i, key in enumerate(corpus.keys()):

    with open('/content/data/html/' + key + '.html','w') as writer:  

        writer.write("<!DOCTYPE html>\n")
        writer.write("<html lang=\"en-US\">\n")
        writer.write("\t<head>\n")
        writer.write("\t\t<meta charset=\"UTF-8\">\n")
        writer.write("\t\t<title>" + key + "</title>\n")
        writer.write("\t</head>\n")
        writer.write("\t<body>\n")
        writer.write("\t\t<main>\n\n")
        writer.write("\t\t\t<p>" + corpus[key] + "</p>\n\n")
        writer.write("\t\t</main>\n")
        writer.write("\t</body>\n")
        writer.write("</html>")

In [ ]:
with open('/content/data/sitemap.xml','w') as writer:    
  
    writer.write('<?xml version="1.0" encoding="UTF-8"?>\n\n')
    writer.write('<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n\n')

    for key, values in corpus.items():
        writer.write('\t<url>\n')
        writer.write('\t\t<loc>https://storage.googleapis.com/robust04-public/' + str(key) + '.html</loc>\n')
        writer.write('\t\t<lastmod>2021-12-7</lastmod>\n')
        writer.write('\t</url>\n\n')
        
    writer.write('\t</urlset>\n\n')

In [ ]:
!gsutil -m cp /content/docx/* gs://robust04-public/

In [ ]:
!gsutil rm -r gs://robust04-public/sitemap.xml
!gsutil cp /content/data/sitemap.xml gs://robust04-public/